In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/My Drive/MAICOGroup/Recommender/test/')
os.getcwd()

'/content/drive/My Drive/MAICOGroup/Recommender/test2'

##Sumarization

In [ ]:
# Import libs & tools
import nltk
import os
import re
import math
import operator

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize

nltk.download('punkt')
nltk.download("stopwords")
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
Stopwords = set(stopwords.words('english'))
wordlemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
# remove special characters funtion (~!@#$%^&*()...)
def remove_special_characters(text):
    regex = r'[^a-zA-Z0-9\s]'
    text = re.sub(regex,'',text)
    return text

In [ ]:
# calculate freqency of words function
def freq(words):
    words = [word.lower() for word in words]
    dict_freq = {}
    words_unique = []
    for word in words:
       if word not in words_unique:
           words_unique.append(word)
    for word in words_unique:
       dict_freq[word] = words.count(word)
    return dict_freq

In [ ]:
# calculate score of sentense function
def sentence_importance(sentence,dict_freq,sentences):
     sentence_score = 0
     sentence = remove_special_characters(str(sentence)) 
     sentence = re.sub(r'\d+', '', sentence)
     pos_tagged_sentence = [] 
     no_of_sentences = len(sentences)
     pos_tagged_sentence = pos_tagging(sentence)
     for word in pos_tagged_sentence:
          if word.lower() not in Stopwords and word not in Stopwords and len(word)>1: 
                word = word.lower()
                word = wordlemmatizer.lemmatize(word)
                sentence_score = sentence_score + word_tfidf(dict_freq,word,sentences,sentence)
     return sentence_score

In [ ]:
# pos tagging function
def pos_tagging(text):
    pos_tag = nltk.pos_tag(text.split())
    pos_tagged_noun_verb = []
    for word,tag in pos_tag:
        if tag == "NN" or tag == "NNP" or tag == "NNS" or tag == "VB" or tag == "VBD" or tag == "VBG" or tag == "VBN" or tag == "VBP" or tag == "VBZ":
             pos_tagged_noun_verb.append(word)
    return pos_tagged_noun_verb

In [ ]:
# calculate TF score function
def tf_score(word,sentence):
    freq_sum = 0
    word_frequency_in_sentence = 0
    len_sentence = len(sentence)
    for word_in_sentence in sentence.split():
        if word == word_in_sentence:
            word_frequency_in_sentence = word_frequency_in_sentence + 1
    tf =  word_frequency_in_sentence/ len_sentence
    return tf

In [ ]:
# calculate IDF score function
def idf_score(no_of_sentences,word,sentences):
    no_of_sentence_containing_word = 0
    for sentence in sentences:
        sentence = remove_special_characters(str(sentence))
        sentence = re.sub(r'\d+', '', sentence)
        sentence = sentence.split()
        sentence = [word for word in sentence if word.lower() not in Stopwords and len(word)>1]
        sentence = [word.lower() for word in sentence]
        sentence = [wordlemmatizer.lemmatize(word) for word in sentence]
        if word in sentence:
            no_of_sentence_containing_word = no_of_sentence_containing_word + 1
    idf = math.log10(no_of_sentences/no_of_sentence_containing_word)
    return idf

In [ ]:
# calculate TFIDF score function
def tf_idf_score(tf,idf):
    return tf*idf

In [ ]:
# calculate TFIDF function
def word_tfidf(dict_freq,word,sentences,sentence):
    word_tfidf = []
    tf = tf_score(word,sentence)
    idf = idf_score(len(sentences),word,sentences)
    tf_idf = tf_idf_score(tf,idf)
    return tf_idf

In [ ]:
def lemmatize_words(words):
    lemmatized_words = []
    for word in words:
       lemmatized_words.append(wordlemmatizer.lemmatize(word))
    return lemmatized_words
def stem_words(words):
    stemmed_words = []
    for word in words:
       stemmed_words.append(stemmer.stem(word))
    return stemmed_words

In [ ]:
# main function
def Summarization_text(file_path, Percentage):
  file = file_path
  file = open(file , 'r')
  text = file.read()
  tokenized_sentence = sent_tokenize(text)
  text = remove_special_characters(str(text))
  text = re.sub(r'\d+', '', text)
  tokenized_words_with_stopwords = word_tokenize(text)
  tokenized_words = [word for word in tokenized_words_with_stopwords if word not in Stopwords]
  tokenized_words = [word for word in tokenized_words if len(word) > 1]
  tokenized_words = [word.lower() for word in tokenized_words]
  tokenized_words = lemmatize_words(tokenized_words)
  word_freq = freq(tokenized_words)
  # input_user = int(input('Percentage of information to retain(in percent):'))
  input_user = Percentage
  no_of_sentences = int((input_user * len(tokenized_sentence))/100)
  print(no_of_sentences)
  c = 1
  sentence_with_importance = {}
  for sent in tokenized_sentence:
      sentenceimp = sentence_importance(sent,word_freq,tokenized_sentence)
      sentence_with_importance[c] = sentenceimp
      c = c+1
  sentence_with_importance = sorted(sentence_with_importance.items(), key=operator.itemgetter(1),reverse=True)
  cnt = 0
  summary = []
  sentence_no = []
  for word_prob in sentence_with_importance:
      if cnt < no_of_sentences:
          sentence_no.append(word_prob[0])
          cnt = cnt+1
      else:
        break
  sentence_no.sort()
  cnt = 1
  for sentence in tokenized_sentence:
      if cnt in sentence_no:
        summary.append(sentence)
      cnt = cnt+1
  summary = " ".join(summary)
  new_file = 'data_summaried/' + file_path.split('/')[1]
  outF = open(new_file,"w")
  outF.write(summary)
  print("\n")
  print("Summary:")
  print(summary)

In [ ]:
# summarization with persentage=50%
text_path = 'data_raw/text1.txt'
Summarization_text(text_path, 50)

7


Summary:
Women education is a catch all term which refers to the state of primary, secondary, tertiary and health education in girls and women. The stronger the roots are the bigger and stronger the tree will be spreading its branches; sheltering and protecting the needy. Women are the soul of a society; a society can well be judged by the way its women are treated. An educated man goes out to make the society better, while an educated woman; whether she goes out or stays at home, makes the house and its occupants better. Women play many roles in a society- mother, wife, sister, care taker, nurse etc. An educated mother will make sure that her children are educated, and will weigh the education of a girl child, same as boys. A society cannot at all progress if its women weep silently.


In [ ]:
import glob
for file_path in glob.glob('data_raw/*.txt'):
  Summarization_text(file_path, 50)

7


Summary:
Women education is a catch all term which refers to the state of primary, secondary, tertiary and health education in girls and women. The stronger the roots are the bigger and stronger the tree will be spreading its branches; sheltering and protecting the needy. Women are the soul of a society; a society can well be judged by the way its women are treated. An educated man goes out to make the society better, while an educated woman; whether she goes out or stays at home, makes the house and its occupants better. Women play many roles in a society- mother, wife, sister, care taker, nurse etc. An educated mother will make sure that her children are educated, and will weigh the education of a girl child, same as boys. A society cannot at all progress if its women weep silently.
2


Summary:
It is the second-most populous country, the seventh-largest country by land area, and the most populous democracy in the world. Bounded by the Indian Ocean on the south, the Arabian Sea o

##Recommendation

In [ ]:
# import libs & tools
import nltk
import glob
import re
import os
import numpy as np
import sys

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import sent_tokenize , word_tokenize

Stopwords = set(stopwords.words('english'))

In [ ]:
# support function
def finding_all_unique_words_and_freq(words):
    words_unique = []
    word_freq = {}
    for word in words:
        if word not in words_unique:
            words_unique.append(word)
    for word in words_unique:
        word_freq[word] = words.count(word)
    return word_freq
def finding_freq_of_word_in_doc(word,words):
    freq = words.count(word)
        
def remove_special_characters(text):
    regex = re.compile('[^a-zA-Z0-9\s]')
    text_returned = re.sub(regex,'',text)
    return text_returned

In [ ]:
class Node:
    def __init__(self ,docId, freq = None):
        self.freq = freq
        self.doc = docId
        self.nextval = None
    
class SlinkedList:
    def __init__(self ,head = None):
        self.head = head

In [ ]:
# load data function
def Load_data(file_folder):
    dict_global = {}
    idx = 1
    files_with_index = {}
    for file in glob.glob(file_folder):
        print(file)
        fname = file
        file = open(file , "r")
        text = file.read()
        text = remove_special_characters(text)
        text = re.sub(re.compile('\d'),'',text)
        sentences = sent_tokenize(text)
        words = word_tokenize(text)
        words = [word for word in words if len(words)>1]
        words = [word.lower() for word in words]
        words = [word for word in words if word not in Stopwords]
        dict_global.update(finding_all_unique_words_and_freq(words))
        files_with_index[idx] = os.path.basename(fname)
        idx = idx + 1
        
    unique_words_all = set(dict_global.keys())
    return unique_words_all, files_with_index

In [ ]:
# Create linklist
def Linklist(unique_words_all, file_folder):
    linked_list_data = {}
    for word in unique_words_all:
        linked_list_data[word] = SlinkedList()
        linked_list_data[word].head = Node(1,Node)
    word_freq_in_doc = {}
    idx = 1
    for file in glob.glob(file_folder):
        file = open(file, "r")
        text = file.read()
        text = remove_special_characters(text)
        text = re.sub(re.compile('\d'),'',text)
        sentences = sent_tokenize(text)
        words = word_tokenize(text)
        words = [word for word in words if len(words)>1]
        words = [word.lower() for word in words]
        words = [word for word in words if word not in Stopwords]
        word_freq_in_doc = finding_all_unique_words_and_freq(words)
        for word in word_freq_in_doc.keys():
            linked_list = linked_list_data[word].head
            while linked_list.nextval is not None:
                linked_list = linked_list.nextval
            linked_list.nextval = Node(idx ,word_freq_in_doc[word])
        idx = idx + 1
    return linked_list_data

In [ ]:
# main function
def Retrieval(files_with_index, linked_list_data):
    query = input('Enter your query:')
    query = word_tokenize(query)
    # print(query)
    connecting_words = []
    cnt = 1
    different_words = []
    for word in query:
        if word.lower() != "and" and word.lower() != "or" and word.lower() != "not":
            different_words.append(word.lower())
        else:
            connecting_words.append(word.lower())
    total_files = len(files_with_index)
    zeroes_and_ones = []
    zeroes_and_ones_of_all_words = []
    for word in (different_words):
        if word.lower() in unique_words_all:
            zeroes_and_ones = [0] * total_files
            linkedlist = linked_list_data[word].head
            # print(word)
            while linkedlist.nextval is not None:
                zeroes_and_ones[linkedlist.nextval.doc - 1] = 1
                linkedlist = linkedlist.nextval
            zeroes_and_ones_of_all_words.append(zeroes_and_ones)
        else:
            # print(word," not found")
            sys.exit
    # print(zeroes_and_ones_of_all_words)
    for word in connecting_words:
        word_list1 = zeroes_and_ones_of_all_words[0]
        word_list2 = zeroes_and_ones_of_all_words[1]
        if word == "and":
            bitwise_op = [w1 & w2 for (w1,w2) in zip(word_list1,word_list2)]
            zeroes_and_ones_of_all_words.remove(word_list1)
            zeroes_and_ones_of_all_words.remove(word_list2)
            zeroes_and_ones_of_all_words.insert(0, bitwise_op);
        elif word == "or":
            bitwise_op = [w1 | w2 for (w1,w2) in zip(word_list1,word_list2)]
            zeroes_and_ones_of_all_words.remove(word_list1)
            zeroes_and_ones_of_all_words.remove(word_list2)
            zeroes_and_ones_of_all_words.insert(0, bitwise_op);
        elif word == "not":
            bitwise_op = [not w1 for w1 in word_list2]
            bitwise_op = [int(b == True) for b in bitwise_op]
            zeroes_and_ones_of_all_words.remove(word_list2)
            zeroes_and_ones_of_all_words.remove(word_list1)
            bitwise_op = [w1 & w2 for (w1,w2) in zip(word_list1,bitwise_op)]
    zeroes_and_ones_of_all_words.insert(0, bitwise_op);
            
    files = []    
    # print(zeroes_and_ones_of_all_words)
    lis = zeroes_and_ones_of_all_words[0]
    cnt = 1
    for index in lis:
        if index == 1:
            files.append(files_with_index[cnt])
        cnt = cnt+1
        
    return files

In [ ]:
# Load model
unique_words_all, files_with_index = Load_data('data_summaried/*')
linked_list_data = Linklist(unique_words_all, 'data_summaried/*')

data_summaried/football.txt
data_summaried/australia.txt
data_summaried/text1.txt
data_summaried/narendra_modi.txt
data_summaried/sneaker.txt
data_summaried/cricket.txt
data_summaried/india.txt
data_summaried/rahul_gandhi.txt
data_summaried/apple.txt


In [ ]:
# run
files_recommend = Retrieval(files_with_index, linked_list_data)
print(files_recommend)

Enter your query:It is the largest country in Oceania and the world's sixth-largest country by total area.
['australia.txt']


In [ ]:
# run
files_recommend = Retrieval(files_with_index, linked_list_data)
print(files_recommend)

Enter your query:shoes primarily designed for sports or other forms of physical exercise
['sneaker.txt']
